[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/weaviate/recipes/blob/main/integrations/llm-agent-frameworks/function-calling/cohere/cohere-query-agent-tool.ipynb)

## Weaviate Query Agent with Cohere

This notebook will show you how to define the Weaviate Query Agent as a tool through Cohere

### Requirements
1. Weaviate Cloud instance (WCD): The Weaviate Query Agent is only accessible through WCD at the moment. You can create a serverless cluster or a free 14-day sandbox [here](https://console.weaviate.cloud/).
1. Install Cohere with `pip install cohere` and grab an API key [here](https://dashboard.cohere.com/api-keys)
1. Install the Weaviate Agents package with `pip install weaviate-agents`
1. You'll need a Weaviate cluster with data. If you don't have one, check out [this notebook](integrations/Weaviate-Import-Example.ipynb) to import the Weaviate Blogs.

### Import libraries

In [ ]:
import weaviate
from weaviate_agents.query import QueryAgent
import os
import json

import cohere

### Initialize client

In [1]:
co = cohere.Client(os.getenv("COHERE_API_KEY"))

### Initialize Weaviate Client and define Query Agent function

In [3]:
def setup_weaviate(weaviate_url, weaviate_api_key, cohere_api_key):
    global weaviate_client, query_agent
    
    # Initialize Weaviate client
    weaviate_client = weaviate.connect_to_weaviate_cloud(
        cluster_url=weaviate_url,
        auth_credentials=weaviate.auth.AuthApiKey(weaviate_api_key),
        headers={"X-Cohere-Api-Key": cohere_api_key},
    )
    
    # connect the query agent to your Weaviate collection(s)
    query_agent = QueryAgent(
        client=weaviate_client,
        collections=["Blogs"] 
    )
        
    print("Weaviate client and Blogs collection initialized successfully")

In [4]:
setup_weaviate(
    weaviate_url=os.getenv("WEAVIATE_URL"),
    weaviate_api_key=os.getenv("WEAVIATE_API_KEY"),
    cohere_api_key=os.getenv("COHERE_API_KEY")
)

Weaviate client and Blogs collection initialized successfully


In [5]:
def send_query_agent_request(query: str) -> str:
    """
    Send a query to the database and get the response.
    
    Args:
        query (str): Any question or query to search for in the database
        
    Returns:
        str: The response from the database containing relevant information
    """
    return query_agent.run(query)

In [8]:
send_query_agent_request("How do I run Weaviate with Docker?")

QueryAgentResponse(original_query='How do I run Weaviate with Docker?', collection_names=['Blogs'], searches=[[QueryResultWithCollection(queries=['run Weaviate with Docker instructions'], filters=[[]], filter_operators='AND', collection='Blogs')]], aggregations=[], usage=Usage(requests=3, request_tokens=7950, response_tokens=620, total_tokens=8570, details=None), total_time=20.148793697357178, aggregation_answer=None, has_aggregation_answer=False, has_search_answer=True, is_partial_answer=False, missing_information=[], final_answer='To run Weaviate with Docker, you will need to use Docker Compose. Here are the steps to get started:\n\n1. **Install Docker and Docker Compose**:\n   - Make sure both the Docker and Docker Compose tools are installed on your system. The installation process can vary depending on your operating system. You can follow the official guides for [Docker Desktop for Mac](https://docs.docker.com/desktop/install/mac-install/), [Docker Desktop for Windows](https://do

### Create tool

In [9]:
functions_map = {
    "send_query_agent_request": send_query_agent_request
}

In [10]:
tools = [
    {
        "name": "send_query_agent_request",
        "description": "Send a query to the database and get the response.",
        "parameter_definitions": {
            "query": {
                "description": "Any question or query to search for in the database",
                "type": "str",
                "required": True
            }
        }
    }
]

### Create a function to run the assistant

In [14]:
model = "command-r-plus"

def run_assistant(message, chat_history=None):
    if chat_history is None:
        chat_history = []

    # Step 1: Get user message
    print(f"Question:\n{message}")
    print("="*50)

    # Step 2: Generate tool calls (if any)    
    response = co.chat(
        message=message,
        model=model,
        tools=tools,
        chat_history=chat_history,
        force_single_step=True
    )

    while response.tool_calls:
        tool_calls = response.tool_calls
        
        if response.text:
            print("Tool plan:")
            print(response.text,"\n")
        print("Tool calls:")
        for call in tool_calls:
            print(f"Tool name: {call.name} | Parameters: {call.parameters}")
        print("="*50)
        
        # Step 3: Get tool results
        tool_results = []
        for tc in tool_calls:
            try:
                tool_call = {"name": tc.name, "parameters": tc.parameters}
                # Create an instance of QueryAgent first if it's a class
                if tc.name == "send_query_agent_request":
                    tool_output = query_agent.run(tc.parameters['query'])
                else:
                    tool_output = functions_map[tc.name](**tc.parameters)
                # Fix: outputs should be an array of objects, not a single object
                tool_results.append({"call": tool_call, "outputs": [{"result": tool_output}]})
            except Exception as e:
                print(f"Error executing tool {tc.name}: {str(e)}")
                # Fix: outputs should be an array of objects, not a single object
                tool_results.append({"call": tool_call, "outputs": [{"error": str(e)}]})
        
        # Step 4: Generate response and citations                
        response = co.chat(
            message="",
            model=model,
            tools=tools,
            tool_results=tool_results,
            chat_history=response.chat_history
        )

        # Append the current chat turn to the chat history
        chat_history = response.chat_history
        
    # Print final response
    print("Final response:")
    print(response.text)
    print("="*50)
    
    # Print citations (if any)
    if response.citations:
        print("Citations:")
        for citation in response.citations:
            print(citation)
        print("\nCited Documents:")
        for document in response.documents:
            print(document)
        print("="*50)
    
    return chat_history

### Query Time

In [15]:
chat_history = run_assistant("Can you teach me how to run Weaviate with Docker?")

Question:
Can you teach me how to run Weaviate with Docker?
Tool calls:
Tool name: send_query_agent_request | Parameters: {'query': 'How to run Weaviate with Docker?'}


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/cohere/core/jsonable_encoder.py:48: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  obj_dict = obj.dict(by_alias=True)


Final response:
Sure, here's a step-by-step guide on how to run Weaviate with Docker:

**Installation Requirements**: 
- Ensure you have Docker and Docker Compose installed. The installation process may vary depending on your operating system (Mac, Windows, or Ubuntu Linux).

**Obtain the Docker Compose File**: 
- Download a `docker-compose.yml` file, which orchestrates the different containers required for Weaviate. You typically don't need to manually obtain individual Weaviate Docker images.

**Set Up and Run**:
- Place the `docker-compose.yml` file in the appropriate directory.
- Use the command `docker compose up -d` to start Weaviate. The `-d` option runs it in detached mode, keeping your command line free.

**Monitoring and Logs**:
- Monitor the logs from Weaviate with the command `docker compose logs -f weaviate`.
- Check the status of running containers with `docker compose ps`.

**Readiness of Weaviate**: 
- To verify that Weaviate is fully up and running, check if a readines